In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle

In [2]:
df = pd.read_csv('../data/interim/train_clean.csv')

In [3]:
df.head()

,ProductName,RtpStateBitfield,IsSxsPassiveMode,AVProductsInstalled,AVProductsEnabled,HasTpm,Platform,Processor,OsVer,OsSuite,...,AppVersion_encoded,Census_OSUILocaleIdentifier_encoded,EngineVersion_encoded,Census_OSInstallLanguageIdentifier_encoded,OrganizationIdentifier_encoded,Census_OSBranch_release_extracted_encoded,OsBuild_encoded,Census_ChassisTypeName_encoded,Census_OSEdition_encoded,Census_OSSkuName_encoded
0,win8defender,7.0,0,1.0,1.0,1,windows10,x64,10.0.0.0,256,...,0,0,0,0,1,0,0,2,1,1
1,win8defender,7.0,0,2.0,1.0,1,windows10,x64,10.0.0.0,768,...,1,0,1,0,1,0,0,1,0,0
2,win8defender,7.0,0,2.0,1.0,1,windows10,x64,10.0.0.0,256,...,0,4,0,4,1,0,0,1,1,1
3,win8defender,7.0,0,2.0,1.0,1,windows10,x64,10.0.0.0,768,...,11,1,1,20,0,8,4,0,2,2
4,win8defender,7.0,0,1.0,1.0,1,windows10,x64,10.0.0.0,256,...,0,19,1,19,0,0,0,0,7,6


In [4]:
df.shape

(89214, 66)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89214 entries, 0 to 89213
Data columns (total 66 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ProductName                                        89213 non-null  object 
 1   RtpStateBitfield                                   88878 non-null  float64
 2   IsSxsPassiveMode                                   89214 non-null  int64  
 3   AVProductsInstalled                                88849 non-null  float64
 4   AVProductsEnabled                                  88849 non-null  float64
 5   HasTpm                                             89214 non-null  int64  
 6   Platform                                           89214 non-null  object 
 7   Processor                                          89209 non-null  object 
 8   OsVer                                              89214 non-null  object 
 9   OsSuit

In [6]:
col_nan = df.isnull().mean()
col_nan.sort_values(ascending=False)[:10]

SmartScreen                                          0.358733
Wdft_RegionIdentifier                                0.032954
Wdft_IsGamer                                         0.032954
Census_InternalBatteryNumberOfCharges                0.030332
Firewall                                             0.010346
Census_TotalPhysicalRAM                              0.009236
Census_IsAlwaysOnAlwaysConnectedCapable              0.007936
Census_SystemVolumeTotalCapacity                     0.006524
Census_PrimaryDiskTotalCapacity                      0.005997
Census_InternalPrimaryDiagonalDisplaySizeInInches    0.005448
dtype: float64

In [7]:
numerical_columns = ['Census_ProcessorCoreCount',
                 'Census_PrimaryDiskTotalCapacity',
                 'Census_SystemVolumeTotalCapacity',
                 'Census_TotalPhysicalRAM',
                 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
                 'Census_InternalPrimaryDisplayResolutionHorizontal',
                 'Census_InternalPrimaryDisplayResolutionVertical',
                 'Census_InternalBatteryNumberOfCharges']
rank_columns = [col for col in df.columns if 'encoded' in col]
numerical_columns = numerical_columns + rank_columns

binary_columns = [col for col in df.columns.tolist() if df[col].nunique() <= 2 if df[col].dtypes != 'O']

categorical_columns = df.select_dtypes('object').columns.tolist()
cat_num_columns = [col for col in df.columns.tolist() if col not in numerical_columns
              if col not in binary_columns if col not in categorical_columns]
tot_columns = numerical_columns + binary_columns + categorical_columns + cat_num_columns
len(tot_columns)

66

In [8]:
len(numerical_columns), len(binary_columns), len(categorical_columns), len(cat_num_columns), len(numerical_columns)+ len(binary_columns) + len(categorical_columns) + len(cat_num_columns)

(32, 10, 17, 7, 66)

In [9]:
col_n_uniq = df[categorical_columns].nunique()
col_n_uniq.sort_values(ascending=False)

Census_MDC2FormFactor               11
OsPlatformSubRelease                 9
Census_PowerPlatformRoleName         9
Census_OSInstallTypeName             9
SkuEdition                           8
SmartScreen                          8
Census_FlightRing                    7
Census_OSWUAutoUpdateOptionsName     6
Census_ActivationChannel             6
Platform                             4
Census_GenuineStateName              4
OsVer                                3
Census_PrimaryDiskTypeName           3
ProductName                          2
Census_OSArchitecture                2
Processor                            2
Census_DeviceFamily                  2
dtype: int64

In [10]:
'HasDetections' in tot_columns

True

In [11]:
target_index = tot_columns.index('HasDetections')
target_name = tot_columns.pop(target_index)

In [12]:
'HasDetections' in tot_columns

False

In [13]:
'HasDetections' in binary_columns

True

In [14]:
idx = binary_columns.index('HasDetections')
binary_columns.pop(idx)

'HasDetections'

In [15]:
'HasDetections' in binary_columns

False

In [16]:
X = df[tot_columns]
y = df[target_name]

In [17]:
#Seperate train and test data
X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size = 0.2,
                                                   random_state = 10)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(71371, 65)
(17843, 65)
(71371,)
(17843,)


In [18]:
y_train.mean(), y_test.mean()

(0.5005114121982318, 0.5026621083898447)

In [19]:
with open('../data/processed/X_train', 'wb') as handle:
    pickle.dump(X_train,handle)
with open('../data/processed/X_test', 'wb') as handle:
    pickle.dump(X_test, handle)
with open('../data/processed/y_train', 'wb') as handle:
    pickle.dump(y_train, handle)
with open('../data/processed/y_test', 'wb') as handle:
    pickle.dump(y_test, handle)